In [ ]:
import re
import pdfplumber
from pprint import pprint
import json

file = "bigUnofficiaTranscript.pdf"
text = ""

honorsRE = re.compile(r'.*honors', re.IGNORECASE)
courseRE = re.compile(r'[A-Z]+\s\d{2,3}')
courseLineRE = re.compile(r'([A-Z]+ \d+) (\w+\s*\w*) ([UGA]*) (\D+?) ([ABCDF-]*)\s?(\d\.\d{3})')
nameRE = re.compile(r'^([A-Za-z]+), ([A-Za-z]+)')
studentChunkRE = re.compile('(Kutztown\nUnofficial Academic Transcript\n.*?)(?=Kutztown\nUnofficial Academic Transcript\n)|(Kutztown\nUnofficial Academic Transcript\n.*)$', re.DOTALL)

# put all text of a pdf into a string
with pdfplumber.open(file) as pdf:
    for page in pdf.pages:
        text += page.extract_text() + "\n"
# print(text)
# split the string across students
allStudentList = re.split(studentChunkRE,text)
allStudentList = filter(None, allStudentList)
allStudentList = list(filter(len,allStudentList))


In [20]:
print(allStudentList[4])
# print(len(allStudentList))
# print(allStudentList

Kutztown
Unofficial Academic Transcript
This is not an official transcript. Courses which are in progress may also be included on this transcript.
Transcript Data
STUDENT INFORMATION
Name Birth Date
Lapenta, Aidan 28-MAY
Current Program
Bachelor of Science
Program College Campus
Information Technology College of Liberal Arts & Sci Kutztown Campus
Major and Department
Information Technology, Computer Sci
& Info Tech
Secondary
Certificate Program
Program College
Cybersecurity Certificate College of Liberal Arts & Sci
Major and Department
Cybersecurity Certificate, Criminal
Justice
DEGREE AWARDED
Sought
Bachelor of Science
Primary Degree
Program College Campus Major
Information Technology College of Liberal Arts & Sci Kutztown Campus Information Technology
Attempt Hours Passed Hours Earned Hours GPA Hours Quality Points GPA
Institution 94.000 94.000 94.000 94.000 376.00 4.00
TRANSFER CREDIT ACCEPTED BY INSTITUTION
Fall 2021: Comm College of the Air Force
Subject Course Title Grade Credit 

In [23]:
for student in allStudentList:
    courseList = []
    newList = []
    honorsList = []
    courseJSON = []
    
    # build a list of all a student's courses
    for line in student.split('\n'):
        if honorsRE.match(line) or courseRE.match(line):
            courseList.append(line)
    print(courseList)
    
    # # join lines where Campus HONORS is on the next line
    # for entry in courseList:
    #     if entry == 'Campus HONORS':
    #         if newList:
    #             newList[-1] += ' ' + entry
    #     else:
    #         newList.append(entry)

    # # build a list of honors courses
    # for course in newList:
    #     if honorsRE.match(course):
    #         honorsList.append(course)

    # cleaned = [s.replace('\n', '') for s in honorsList]

    # # parse out a course into pieces of info
    # for course in cleaned:
    #     if courseLineRE.match(course):
    #         course = courseLineRE.search(course)
    #         if course:
    #             courseCode = course.group(1)
    #             title = course.group(4)
    #             grade = course.group(5)
    #             creditHours = course.group(6)
    #             courseDict = {courseCode: {"title": title, "grade": grade, "credits": creditHours}}
    #             courseJSON.append(courseDict)

    # # get student names and add it to json
    # for line in text.split('\n'):
    #     if nameRE.match(line):
    #         line = nameRE.search(line)
    #         if line:
    #             lastName = line.group(1)
    #             firstName = line.group(2)
    #             studentJSON = {firstName + ' ' + lastName: courseJSON}
    # y = json.dumps(studentJSON)

    # pprint(studentJSON)

['BIO 104 PRINCIPLES OF BIOLOGY CR 4.000 0.00', 'BIO 120 ANATOMY & PHYSIO I CR 4.000 0.00', 'SPA 101 ELEMENTARY SPANISH I P 3.000 0.00', 'SPA 102 ELEMENTARY SPANISH II P 3.000 0.00', 'CMP 100 Kutztown UG Effective Composition A 3.000 12.00', 'COM 10 Kutztown UG FUNDAMENTALS OF ORAL COMM - A- 3.000 11.01', 'Campus HONORS', 'FYS 100 Kutztown UG FIRST YEAR SEMINAR - FOLLOW THE A 3.000 12.00', 'MAT 105 Kutztown UG College Algebra A 3.000 12.00', 'PSY 11 Kutztown UG GENERAL PSYCHOLOGY - HONORS A 3.000 12.00', 'CMP 200 Kutztown UG RESEARCH AND COMPOSITION - A 3.000 12.00', 'Campus HONORS', 'COM 130 Kutztown UG Interpersonal Communication A 3.000 12.00', 'ENG 229 Kutztown UG Literature and Psychology A 3.000 12.00', 'PSY 125 Kutztown UG Life Span Development A 3.000 12.00', 'PSY 200 Kutztown UG Statistics for the Social and A 3.000 12.00', 'ANT 105 Kutztown UG Classical Archaeology B+ 3.000 9.99', 'COM 261 Kutztown UG INTRO/HEALTH COMMUNICATN - A 3.000 12.00', 'Campus HONORS', 'PSY 250 Kutzto